In [ ]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

In [ ]:
level = 2
setup = 'oversigma_10tissue'
#label = 'disease_type'
label='primary_site'
#label = 'uniq'
L = 3
#labels = ['primary_site', 'disease_type']
labels = ['primary_site', 'secondary_site']
#directory = "results/hSBM/%s"%setup
directory="gtex/hsbm/%s"%setup
df = pd.read_csv("%s/mainTable.csv"%(directory), index_col=[0], header=[0])
df.head()

In [ ]:
data=df.T.values[:1183]

In [ ]:
data.shape

In [ ]:
dend = shc.dendrogram(shc.linkage(data, method='ward'))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')  
out = cluster.fit_predict(data)

In [ ]:
from hsbmpy import define_labels, get_cluster_given_l

In [ ]:
true_labels, predicted_labels = define_labels(get_cluster_given_l(2,directory),pd.read_csv("%s/files.dat"%directory, index_col=[0]), label='primary_site')

In [ ]:
np.shape(out)

In [ ]:
np.shape(predicted_labels)

In [ ]:
np.shape(true_labels)

In [ ]:
from sklearn.metrics import homogeneity_completeness_v_measure

In [ ]:
homogeneity_completeness_v_measure(true_labels,predicted_labels)

In [ ]:
df_files = pd.read_csv("%s/files.dat"%directory, index_col=[0])

In [ ]:
from hsbmpy import get_file

In [ ]:
scores = {}
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}

In [ ]:
for x in [1183, 268, 34, 10]:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(data)
    true_out = []
    for sample in df.columns.values[0:1183]:
        true_out.append(get_file(sample, df_files)['primary_site'])
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierarchical']['h'].append(score[0])
    scores['hierarchical']['c'].append(score[1])
    scores['hierarchical']['V'].append(score[2])

In [ ]:
scores

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=100,random_state=42)
topics = lda.fit_transform(data)

In [ ]:
cm = sns.clustermap(topics)

In [ ]:
cluster.fit_predict(topics)

In [ ]:
scores['lda']={
    'h':[],
    'c':[],
    'V':[]
}
for x in [1183, 268, 34, 10]:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(topics)
    true_out = []
    for sample in df.columns.values[0:1183]:
        true_out.append(get_file(sample, df_files)['primary_site'])
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['lda']['h'].append(score[0])
    scores['lda']['c'].append(score[1])
    scores['lda']['V'].append(score[2])

In [ ]:
scores['lda']